<a href="https://colab.research.google.com/github/Oksana0020/Hokku/blob/main/Hokku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Genetic Algorithm Assignment Hokku

## The Problem         **(~30%)**

*   Description of the problem

*   Discussion of the suitablity of Genetic Algorithms

*   Complexity of the problem  (Overall marks allocated based on ..)

# The problem and the cost function   **(~20%)**

# The Individual **(~30%)**


*   Chromosone
*   Crossover
*   Mutation

## Discussion and justification on the approaches taken for the above

## Running the algorithm  **(~10%)**

*   Parameter choices
*   Modifications (if any) to run_genetic
*   Rationale for the above

In [ ]:
#  Parameter class here

In [ ]:
# Run Genetic method here

In [ ]:
#  Running of the algorithm with outputs here

In [ ]:
#  If changes to params or reruns of iterations dont overwrite, create more cells and copy code down to show evolution of final solution

Results and conclusions (~10%)

### Hokku Generator using Genetic Algorithms

**A hokku** is a very short poem written in three lines following 5–7–5 syllable pattern. In my project, a hokku is generated text built by choosing words from vocabulary and arranging them into 3 lines following syllables pattern 5–7–5.

**A chromosome** is one candidate hokku solution. It is encoded as an array of integers. Each integer corresponds to an index in the vocabulary.The chromosome length is fixed and represents word slots for all three lines of the poem.When decoded, the chromosome maps to words that form the hokku text.

**The cost function** is a scoring rule that measures how bad a candidate hokku is. Lower cost means a better poem, with the ideal goal being to approach zero cost. The cost function is built incrementally and enforces the following rules:

- *Syllable mismatch penalty*
Penalize deviation from the 5–7–5 syllable pattern.

- *Marker rule*
Exactly one marker must be present in the poem.
Later stages prefer the marker at the end of line 1 or line 2.
This enables a later *ending rule* (do not end the poem with a marker or connector).

- *Repetition penalty*
Penalize repeated words within the same poem.

- *Season-word rule*
Require at least one season word per poem (added at a later stage).

- *Grammar rules*
Examples:
1)Preferred word order such as
adj noun verb or adj noun verb adv
2)At least two adverbs within one poem
3)Avoid ending the poem with short connector / preposition words

**Mutation** introduces random variation into an individual chromosome so the Genetic Algorithm can explore new possibilities. During mutation, a word may be replaced by another word from the same category (for example, replacing one noun with a different noun).It allows new word combinations to appear.

**Crossover** combines two parents to produce children: parent1, parent2 are making children by blending genes. Crossover mixes parents’ word choices. Kids  inherit word positions from each parent.Crossover reuses good partial solutions

### Hokku Generator using Genetic Algorithms

**A hokku** is a very short poem written in three lines following 5–7–5 syllable pattern. In my project, a hokku is generated text built by choosing words from vocabulary and arranging them into 3 lines following syllables pattern 5–7–5.

**A chromosome** is one candidate hokku solution. It is encoded as an array of integers. Each integer corresponds to an index in the vocabulary.The chromosome length is fixed and represents word slots for all three lines of the poem.When decoded, the chromosome maps to words that form the hokku text.

**The cost function** is a scoring rule that measures how bad a candidate hokku is. Lower cost means a better poem, with the ideal goal being to approach zero cost. The cost function is built incrementally and enforces the following rules:

- *Syllable mismatch penalty*
Penalize deviation from the 5–7–5 syllable pattern.

- *Marker rule*
Exactly one marker must be present in the poem.
Later stages prefer the marker at the end of line 1 or line 2.
This enables a later *ending rule* (do not end the poem with a marker or connector).

- *Repetition penalty*
Penalize repeated words within the same poem.

- *Season-word rule*
Require at least one season word per poem (added at a later stage).

- *Grammar rules*
Examples:
  - 1)Preferred word order such as
adj noun verb or adj noun verb adv
  - 2)At least two adverbs within one poem
  - 3)Avoid ending the poem with short connector / preposition words

**Mutation** introduces random variation into an individual chromosome so the Genetic Algorithm can explore new possibilities. During mutation, a word may be replaced by another word from the same category (for example, replacing one noun with a different noun, or one season word with another season word).It allows new word combinations to appear.

**Crossover** combines two parents to produce children: parent1, parent2 are making children by blending genes. Crossover mixes parents’ word choices. Kids  inherit word positions from each parent.Crossover reuses solutions

**Example of Hokku**


Through my shanks are thin

I go where flowers blossom,

Yoshino Mountain.


*Basho*

Example is taken from "One Hundred Famous Haiku" by Daniel C. Buchanan
https://thehaikufoundation.org/omeka/files/original/50a61193bc1caf7a3c86b9acc96d4b44.pdf

In [67]:
import numpy as np
from copy import deepcopy

#Vocabulary
for now consists of markers which are punctuation symbols, nouns, verbs, adjectives, adverbs and connectors (preposiitons, conjunctions)

Need to add season words so cost function that there needs to be at least 1 season word can be introduced

In [16]:
WORD_DATA = [
    # markers
    {"word": "—", "syllable": 0, "category": "marker"},
    {"word": "...", "syllable": 0, "category": "marker"},
    {"word": "!", "syllable": 0, "category": "marker"},
    {"word": "?", "syllable": 0, "category": "marker"},
    {"word": ",", "syllable": 0, "category": "marker"},
    {"word": ".", "syllable": 0, "category": "marker"},
    {"word": ";", "syllable": 0, "category": "marker"},
    {"word": ":", "syllable": 0, "category": "marker"},
    {"word": "@", "syllable": 0, "category": "marker"},
    {"word": "¿", "syllable": 0, "category": "marker"},

    # nouns
    {"word": "pond", "syllable": 1, "category": "noun"},
    {"word": "frog", "syllable": 1, "category": "noun"},
    {"word": "river", "syllable": 2, "category": "noun"},
    {"word": "stone", "syllable": 1, "category": "noun"},
    {"word": "leaf", "syllable": 1, "category": "noun"},
    {"word": "shadow", "syllable": 2, "category": "noun"},
    {"word": "silence", "syllable": 2, "category": "noun"},
    {"word": "moon", "syllable": 1, "category": "noun"},
    {"word": "wind", "syllable": 1, "category": "noun"},
    {"word": "lantern", "syllable": 2, "category": "noun"},

    # verbs
    {"word": "jumps", "syllable": 1, "category": "verb"},
    {"word": "drifts", "syllable": 1, "category": "verb"},
    {"word": "falls", "syllable": 1, "category": "verb"},
    {"word": "whispers", "syllable": 2, "category": "verb"},
    {"word": "shines", "syllable": 1, "category": "verb"},
    {"word": "turns", "syllable": 1, "category": "verb"},
    {"word": "melts", "syllable": 1, "category": "verb"},
    {"word": "settles", "syllable": 2, "category": "verb"},
    {"word": "wanders", "syllable": 2, "category": "verb"},
    {"word": "sleeps", "syllable": 1, "category": "verb"},

    # adjectives
    {"word": "old", "syllable": 1, "category": "adj"},
    {"word": "silent", "syllable": 2, "category": "adj"},
    {"word": "soft", "syllable": 1, "category": "adj"},
    {"word": "bright", "syllable": 1, "category": "adj"},
    {"word": "cold", "syllable": 1, "category": "adj"},
    {"word": "dark", "syllable": 1, "category": "adj"},
    {"word": "misty", "syllable": 2, "category": "adj"},
    {"word": "gentle", "syllable": 2, "category": "adj"},
    {"word": "lonely", "syllable": 2, "category": "adj"},
    {"word": "empty", "syllable": 2, "category": "adj"},

    # connectors
    {"word": "in", "syllable": 1, "category": "other"},
    {"word": "of", "syllable": 1, "category": "other"},
    {"word": "to", "syllable": 1, "category": "other"},
    {"word": "from", "syllable": 1, "category": "other"},
    {"word": "with", "syllable": 1, "category": "other"},
    {"word": "without", "syllable": 2, "category": "other"},
    {"word": "through", "syllable": 1, "category": "other"},
    {"word": "under", "syllable": 2, "category": "other"},
    {"word": "near", "syllable": 1, "category": "other"},
    {"word": "while", "syllable": 1, "category": "other"},

    # adverbs
    {"word": "slowly", "syllable": 2, "category": "adv"},
    {"word": "softly", "syllable": 2, "category": "adv"},
    {"word": "silently", "syllable": 3, "category": "adv"},
    {"word": "brightly", "syllable": 2, "category": "adv"},
    {"word": "deeply", "syllable": 2, "category": "adv"},
    {"word": "lightly", "syllable": 2, "category": "adv"},
    {"word": "suddenly", "syllable": 3, "category": "adv"},
    {"word": "quietly", "syllable": 3, "category": "adv"},
    {"word": "still", "syllable": 1, "category": "adv"},
    {"word": "again", "syllable": 2, "category": "adv"},

    #season words to be added
]

VOCABULARY = [w["word"] for w in WORD_DATA]
SYLLABLE   = {w["word"]: w["syllable"] for w in WORD_DATA}
CATEGORY   = {w["word"]: w["category"] for w in WORD_DATA}
MARKER_SET = {w for w in VOCABULARY if CATEGORY[w] == "marker"}

In [68]:
VOCABULARY[50]

'slowly'

In [69]:
CATEGORY["frog"]

'noun'

In [70]:
SYLLABLE["frog"]

1

Simple checkup for vocabulary

In [21]:
from collections import Counter

category_counts = Counter(CATEGORY[w] for w in VOCABULARY)
print("Category counts:", category_counts)
print("Total vocabulary size:", len(VOCABULARY))
print("Markers are the following:", sorted(MARKER_SET))
print("Syllables in 'silently':", SYLLABLE["silently"])
print("Syllables in 'misty':", SYLLABLE["misty"])



Category counts: Counter({'marker': 10, 'noun': 10, 'verb': 10, 'adj': 10, 'other': 10, 'adv': 10})
Total vocabulary size: 60
Markers are the following: ['!', ',', '.', '...', ':', ';', '?', '@', '¿', '—']
Syllables in 'silently': 3
Syllables in 'misty': 2


#Adding chromosome and some helpers

slots_per_line does not mean syllables, it means how many words Genetic Algorithm is allowed to choose per line.

chromosome length is:
number_of_genes = 3 * slots_per_line

In [22]:
slots_per_line = 4
number_of_genes = 3 * slots_per_line

#Convert a numeric chromosome into three lines of words
def chromosome_to_lines(chromosome, slots_per_line: int):
    words = [VOCABULARY[int(i)] for i in chromosome]
    return [
        words[0:slots_per_line],
        words[slots_per_line:2 * slots_per_line],
        words[2 * slots_per_line:3 * slots_per_line],
    ]

#Calculate total syllables of line
def line_syllables(words):
    return sum(SYLLABLE[w] for w in words)

def format_hokku(lines):
    formatted_lines = []
    for line in lines:
        out = []
        for w in line:
            if w in MARKER_SET:
                if out:
                    out[-1] = out[-1] + w
                else:
                    out.append(w)
            else:
                out.append(w)
        formatted_lines.append(" ".join(out))
    return "\n".join(formatted_lines)

## Checking chromosome decoding using raw integer indices.

In [31]:
chromosome = [
    50, 1, 11, 0,     # line 1
    12, 20, 32, 5,   # line 2
    18, 14, 49, 2,   # line 3
]

lines = chromosome_to_lines(chromosome, slots_per_line)

print("Decoded lines:", lines)
print()
print("Formatted hokku:\n")
print(format_hokku(lines))
print()
print("Syllables per line:", [line_syllables(line) for line in lines])

Decoded lines: [['slowly', '...', 'frog', '—'], ['river', 'jumps', 'soft', '.'], ['wind', 'leaf', 'while', '!']]

Formatted hokku:

slowly... frog—
river jumps soft.
wind leaf while!

Syllables per line: [3, 4, 3]


## Cost function version 1 (penalty for syllable mismatch)

- A hokku should follow the 5–7–5 syllable pattern.
- The cost is the total deviation from this pattern.
- Lower cost is better, and cost reaches 0 when syllables match perfectly.


In [71]:
def hokku_cost_v1(chromosome, slots_per_line: int) -> float:
    target = [5, 7, 5]

    lines = chromosome_to_lines(chromosome, slots_per_line)
    syllables = [line_syllables(line) for line in lines]

    cost = 0.0
    for i in range(3):
        cost += abs(syllables[i] - target[i])

    return float(cost)


### Checkup of cost function (version 1)

Cost function computes the syllables per line and the resulting cost.
A perfect hokku (5–7–5) would give cost = 0.


In [72]:
syllables = [line_syllables(line) for line in lines]
print("Syllables per line:", syllables)
print("Cost is:", hokku_cost_v1(chromosome, slots_per_line))


Syllables per line: [3, 4, 3]
Cost is: 7.0


So initial example has cost 7.0

Syllables per line: [3, 4, 3]
Cost is: 7.0

Before implementing Genetic Algorithm, I plan to evaluate initial cost function
on many randomly generated chromosomes. When Genetic Algorithm works correctly, it should produce better results than this random function.

#Randomly generated chromosome

In [73]:
import random

def random_chromosome(number_of_genes: int):
    chromosome = []
    for _ in range(number_of_genes):
        gene = random.randint(0, len(VOCABULARY) - 1)
        chromosome.append(gene)
    return chromosome


best_cost = float("inf") #possible infinity
best_chromosome = None
trials = 1000

for _ in range(trials):
    chrom = random_chromosome(number_of_genes)
    c = hokku_cost_v1(chrom, slots_per_line)

    if c < best_cost:
        best_cost = c
        best_chromosome = chrom


best_lines = chromosome_to_lines(best_chromosome, slots_per_line)

print("Trials:", trials)
print("Best cost found:", best_cost)
print("Best random hokku:\n")
print("====================")
print(format_hokku(best_lines))
print("====================")
print("Syllables per line:", [line_syllables(line) for line in best_lines])

Trials: 1000
Best cost found: 0.0
Best random hokku:

lonely¿ moon misty
lightly shadow through lantern
cold with lantern cold
Syllables per line: [5, 7, 5]


Random search rarely produces poems that satisfy the 5–7–5 pattern. Genetic A has to outperform random search

##Genetic Algorithm version1

Implement a Genetic Algorithm version1 to optimize hokku syllable structure 5-7-5


- an Individual is a chromosome (array of integer word indices),
- the cost is the syllable mismatch penalty only for now,
- mutation randomly replaces some indices,
- crossover combines two parents at a random cut point.

Goal: reach cost = 0 (perfect 5–7–5), faster than random sampling.

In [74]:
len(VOCABULARY)

60

In [75]:
class problem:
    def __init__(self, slots_per_line: int = 4):
        self.slots_per_line = int(slots_per_line)
        self.number_of_genes = 3 * self.slots_per_line
        self.min_gene_value = 0
        self.max_gene_value = len(VOCABULARY) - 1

        self.cost_function = lambda chrom: hokku_cost_v1(chrom, self.slots_per_line)

In [76]:
class Parameters:
    def __init__(self):
        self.population_size = 1000
        self.generations = 300
        self.child_rate = 0.6
        self.mutation_rate = 0.15 #(15% chance to mutate, 85% chance to stay the same)

class Individual one solution to genetic algorithm,

chromosome list of numbers,

not to go out of range so need min and max, the possible min value is 0 because my list starts with index 0, max value len(vocab)-1

def crossover i will concatenate first part from parent1 and second part from parent2

def mutate - small random changes, loop through each geen and decide weatehr to muatte

In [77]:
class Individual:
    def __init__(self, prob: problem):
        self.chromosome = np.random.randint(
            prob.min_gene_value,
            prob.max_gene_value + 1,
            size=prob.number_of_genes,
        )
        self.cost = prob.cost_function(self.chromosome)

    def crossover(self, other: "Individual") -> np.ndarray:
        n = len(self.chromosome)
        point = np.random.randint(1, n)
        return np.concatenate([self.chromosome[:point], other.chromosome[point:]]).astype(int)

    def mutate(self, prob: problem, mutation_rate: float):
        for i in range(len(self.chromosome)):
            if np.random.rand() < mutation_rate:
                self.chromosome[i] = np.random.randint(prob.min_gene_value, prob.max_gene_value + 1)

###Genetic Algorithm runner

In [78]:
def choose_two_parents(population: list[Individual]):
    i1 = np.random.randint(0, len(population))
    i2 = np.random.randint(0, len(population))
    if i1 == i2:
        return choose_two_parents(population)
    return population[i1], population[i2]


def run_genetic(prob: problem, params: Parameters):
    population = [Individual(prob) for _ in range(params.population_size)]
    population.sort(key=lambda ind: ind.cost)

    best = deepcopy(population[0])
    children_per_generation = int(params.population_size * params.child_rate)

    for gen in range(params.generations):
        children = []

        while len(children) < children_per_generation:
            parent1, parent2 = choose_two_parents(population)

            child_chrom = parent1.crossover(parent2)

            child = Individual(prob)
            child.chromosome = child_chrom
            child.mutate(prob, params.mutation_rate)
            child.cost = prob.cost_function(child.chromosome)

            children.append(child)

        population.extend(children)
        population.sort(key=lambda ind: ind.cost)
        population = population[:params.population_size]

        if population[0].cost < best.cost:
            best = deepcopy(population[0])

        # progress print (simple)
        if gen % 20 == 0:
            print("Generation:", gen, "Best cost:", best.cost)

        if best.cost == 0:
            break

    return best

First resuts after running Genetic Algorithm

In [79]:
prob = problem(slots_per_line=slots_per_line)
params = Parameters()

best = run_genetic(prob, params)
best_lines = chromosome_to_lines(best.chromosome, slots_per_line)

print("\nBest GA cost:", best.cost)
print("Best GA hokku:\n")
print("====================")
print(format_hokku(best_lines))
print("====================")
print("Syllables per line:", [line_syllables(line) for line in best_lines])

Generation: 0 Best cost: 0.0

Best GA cost: 0.0
Best GA hokku:

river near old near
slowly shadow wind whispers
turns wanders pond falls
Syllables per line: [5, 7, 5]


#Cost Function (version 2)

Marker Rule means that exactly one marker must appear within a poem
Poems without a marker or with multiple markers
are penalized.

(next step for markers will be - prefer having marker at end of line 1 or 2

and also implementing ending rule (do not end poem with marker so no marker at end of line 3))

In Genetic Algorithms there are several ways to choose parents for reproduction. The two most common ones are:

- Tournament Selection

- Roulette Wheel Selection

Need to look at different ways of choosing parent
population around 200, then I change to 1000

At the end

## Results and conclusions    **(~10%)**

chromosome, generate justify why it is sollution,
simple cost function
crossover


Game: create board, display board, all possible moves